In [36]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from scipy.fft import fft, ifft
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
import plotly.graph_objs as go
from dash import Dash, dcc, html, Input, Output, ctx
from datetime import timedelta
import random
import joblib

In [37]:
# Establecer una semilla aleatoria
seed_value = 24
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [38]:

# Función para calcular el ciclo circadiano utilizando la Transformada de Fourier
def extract_circadian_cycle(data, sample_rate=24, z=1):
    
    fft_result = fft(data)
    n = len(data)
    freq = np.fft.fftfreq(n, d=1/sample_rate)
    
    # Mantener solo los armónicos en el rango [-z, z]
    fft_filtered = np.zeros_like(fft_result, dtype=complex)
    for k in range(-z, z + 1):
        idx = np.argmin(np.abs(freq - k))  # Encuentra el índice más cercano a la frecuencia k
        fft_filtered[idx] = fft_result[idx]
    
    # Reconstrucción de la señal para obtener el ciclo circadiano
    return np.real(ifft(fft_filtered))
      

In [39]:
def predecir_etiquetas(df,X_test_reshaped,model):
    
    try:
        # Predecir las etiquetas
        y_pred_proba = model.predict(X_test_reshaped)
        y_pred_classes = np.argmax(y_pred_proba, axis=1)
        #print("Clases predichas:", y_pred_classes)
        # Identificar horas con niveles de estrés alerta y peligro
        niveles_estres = ['Normal', 'Leve', 'Alto']
        horas_estres = [(df['datetime'].iloc[i+24], niveles_estres[clase])
                            for i, clase in enumerate(y_pred_classes) if clase > 0]
        # Convert horas_estres to datetime
        print("Horas con niveles de estrés leve y alto:", horas_estres)
    except Exception as e:
        print(f"Error en la predicción: {e}")
        horas_estres = []
    
    
    return horas_estres

In [40]:
def predecir_etiquetas_futura(df_predichos,X_test,model,timesteps):
    
    try:
        # predecccion teimpo real
        ventana_actual = X_test[-timesteps:]  # Últimas 24 horas
        nueva_prediccion = model.predict(ventana_actual.reshape(1,timesteps, -1))
        nueva_prediccion_classes = np.argmax(nueva_prediccion, axis=1)
        # Identificar horas con niveles de estrés alerta y peligro
        niveles_estres = ['Normal', 'Leve', 'Alto']
        horas_estres_f = [(df_predichos['Fecha'].iloc[i], niveles_estres[clase])
                            for i, clase in enumerate(nueva_prediccion_classes)]
        # Convert horas_estres to datetime
        print("Horas con niveles de estrés leve y alto:", horas_estres_f)
    except Exception as e:
        print(f"Error en la predicción: {e}")
        horas_estres_f = []
    
    return horas_estres_f

In [41]:
def predecir_futuro(model, X_test_reshaped,timesteps, dias_a_predecir):
    """Realiza predicciones para días futuros """
    predicciones_futuras = []
    
    ultimo_valor =X_test_reshaped[-1].reshape((1, timesteps, X_test_reshaped.shape[2]))

    for _ in range(dias_a_predecir* 24):
        prediccion = model.predict(ultimo_valor,verbose=0)
        predicciones_futuras.append(prediccion[0][0])  # Guarda solo el valor escalar
        prediccion_reshaped = np.broadcast_to(prediccion[:, np.newaxis, :], (1, 1, X_test_reshaped.shape[2])) #se repite la prediccion n_features veces para que tenga la misma dimension que ultimo valor
        ultimo_valor = np.concatenate((ultimo_valor[:, 1:, :], prediccion_reshaped), axis=1)

    return np.array(predicciones_futuras)

In [42]:

# Función para generar datos reales
def cargar_datos_reales(df,fecha_inicio,activity_column):
    """Carga datos reales para una fecha dada """   
    fecha_minima = df['datetime'].min()
    fecha_actual = fecha_inicio 
    
    while fecha_actual > fecha_minima and fecha_actual not in df['datetime'].values:
        fecha_actual -= pd.Timedelta(days=1)  # Retroceder un día
    
    #si retrocede mas alla de la fecha minima, se toma la fecha minima
    if fecha_actual < fecha_minima:
        fecha_actual = fecha_minima
    
    print(fecha_actual)
       
    df_filtrados = df.loc[(df['datetime'] >= fecha_actual)]
    
    fechas_reales = df_filtrados['datetime']
    actividad_real = df_filtrados[activity_column]
    ciclo_circadiano = extract_circadian_cycle(actividad_real.values)
    prediccion = df_filtrados['nivel_actividad_p']
    ciclo_c_pred  = extract_circadian_cycle(prediccion.values)
   
    return pd.DataFrame({'Fecha': fechas_reales, 'Actividad Real': actividad_real,'Ciclo':ciclo_circadiano,'Prediccion': prediccion,'Ciclo_p':ciclo_c_pred})

# Función para generar datos predichos
def cargar_datos_predichos(model_n_act,fecha_inicio,X_test_reshaped,timesteps):
    dias_a_predecir = 1
    fecha_inicio += timedelta(hours=1)
    #y_pred= model_n_act.predict(X_test_reshaped)
    fechas_predichas = pd.date_range(start=fecha_inicio, periods=24 * dias_a_predecir, freq='h')
    prediccion = predecir_futuro(model_n_act, X_test_reshaped, timesteps ,dias_a_predecir)
    ciclo_c_pred  = extract_circadian_cycle(prediccion)
    
    return pd.DataFrame({'Fecha': fechas_predichas, 'Actividad Real': None, 'Ciclo':None, 'Prediccion': prediccion,'Ciclo_p':ciclo_c_pred})

# Función para agregar anotaciones de estrés al gráfico
def agregar_anotaciones(fig, df_completo, horas_estres, colmna,colors):
    # Filtrar solo las horas de estrés que están presentes en df_completo
    for hora, nivel_estres in horas_estres:
        # Verificar si la hora está en el DataFrame df_completo
        if hora in df_completo['Fecha'].values:
            # Encontrar el índice de la hora en df_completo
            idx = df_completo[df_completo['Fecha'] == hora].index[0]
            
            # Obtener el valor de la actividad real en esa hora
            actividad_real = df_completo.iloc[idx][colmna]
            
            # Agregar anotación al gráfico
            fig.add_annotation(
                x=hora,
                y=actividad_real,
                text=nivel_estres,
                showarrow=True,
                arrowhead=2,
                ax=0,
                ay=-40,  # Ajuste vertical de la anotación
                font=dict(size=12, color=colors),
                arrowcolor='black',
            )

In [ ]:
# -------------------- PARTE 1: Llamado de datos-------------------- #

# Parámetros
nombre_csv = 4003  # Nombre base del archivo CSV
timesteps = 24  # Número de pasos temporales
# Cargar el modelo guardado
try:
    model_n_estres = load_model(f'entrenamientos/{nombre_csv}_modelo_entrenado.h5')
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    exit()
    
# Cargar el modelo de actividad guardado
try:
    model_n_act = load_model(f'entrenamientos/{nombre_csv}_modelo_actividad.h5')
except Exception as e:
    print(f"Error al cargar el modelo del nivel de actividad: {e}")
    exit()       

# Cargar los datos
try:
    df = pd.read_csv(f'Datos_prueba2/{nombre_csv}_p.csv')
    df['datetime'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'].astype(str) + ':00:00')
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    # Seleccionar características para predicción
    X_test = df[['period eating','period other','period resting','period rumination']].values
    
except Exception as e:
    print(f"Error al cargar el archivo CSV: {e}")
    exit()

# Validar si hay suficientes datos para estandarización
try:
    scaler_cargado = joblib.load(f'scalers/{nombre_csv}_scaler.pkl')
    X_test_scaled = scaler_cargado.transform(X_test)
except Exception as e:
    print(f"Error al escalar los datos: {e}")
    exit()
   
#Validar que hay suficientes datos para crear secuencias
if len(X_test) > timesteps:
    # Crear secuencias temporales
    X_test_reshaped = np.array([X_test_scaled[i:i + timesteps] for i in range(len(X_test_scaled) - timesteps)])
    #print("Forma de X_test_reshaped:", X_test_reshaped.shape)  
else:
    print("No hay suficientes datos para generar secuencias de 24 timesteps.")
 
if X_test_reshaped.size > 0:
    df['nivel_actividad_p'] = np.nan
    prediccion_n_actividad = model_n_act.predict(X_test_reshaped,verbose=0)
    df.loc[timesteps:timesteps+len(prediccion_n_actividad)- 1, 'nivel_actividad_p'] = prediccion_n_actividad    
else:
    print("Advertencia: No hay suficientes secuencias para predicción.")
# Realizar la predicción   

   
# -------------------- PARTE 2: DASHBOARD INTERACTIVO -------------------- #

# Crear la app de Dash
app = Dash(__name__) 

# Generar datos iniciales
fecha_inicial = df['Fecha'].max() - pd.Timedelta(days=2)
df_reales = cargar_datos_reales(df, fecha_inicial,'nivel_actividad')
df_predichos = cargar_datos_predichos(model_n_act, df_reales['Fecha'].max(),X_test_reshaped,timesteps)
fecha_separacion=df_reales['Fecha'].max()

# Concatenar datos reales y predichos
df_completo = pd.concat([df_reales, df_predichos], ignore_index=True)
# Extracción de etiqueta etiquetas reales y predichas
horas_estres_r = df[df['nivel_estres'] != 'Normal'][['datetime', 'nivel_estres']].values
horas_estres_pred = predecir_etiquetas(df,X_test_reshaped,model_n_estres)
horas_estres_f=predecir_etiquetas_futura(df_predichos,X_test,model_n_estres,timesteps)

#print("Horas con niveles de estrés leve y alto:", horas_estres_r)
#print("Horas con niveles de estrés leve y alto:", horas_estres_pred)

# Layout de la app
app.layout = html.Div([
    html.H1("Monitoreo del Nivel de Actividad y Predicción", style={'textAlign': 'center'}),
    # Indicador de carga para la gráfica
    dcc.Loading(
        id="loading",
        type="circle",  # Puedes cambiar a 'dot' o 'default'
        children=[dcc.Graph(id='grafica-continuada')]
    ),
    html.Button("Cargar Más Datos", id='boton-cargar', n_clicks=0, style={'margin': '10px'}),
    dcc.Checklist(
        id='toggle-anotaciones',
        options=[{'label': 'Equiquetas - Actividad Real', 'value': 'ON'}],
        value=[]
    ),
    dcc.Checklist(
        id='toggle-anotaciones2',
        options=[{'label': 'Equiquetas - Predichas', 'value': 'ON'}],
        value=[]
    )
])

# Callback para actualizar el gráfico
@app.callback(
    Output('grafica-continuada', 'figure'),
    [Input('boton-cargar', 'n_clicks'),
    Input('toggle-anotaciones', 'value'),
    Input('toggle-anotaciones2', 'value')]
    
)

def actualizar_grafico(n_clicks,activar_anotaciones, activar_anotaciones2):
    global df_completo, fecha_separacion
    # Detectar qué input activó el callback
    triggered_id = ctx.triggered_id

    # Cargar más datos si se presiona el botón
    if triggered_id == 'boton-cargar' and n_clicks:
        # Generar nuevos datos
        fecha_inicio_nueva = df_completo['Fecha'].min() - pd.Timedelta(days=7)
        nuevos_datos = cargar_datos_reales(df,fecha_inicio_nueva,'nivel_actividad')
        fecha_separacion=df_reales['Fecha'].max()
        # Concatenar y limpiar
        df_completo = pd.concat([nuevos_datos, df_completo], ignore_index=True).drop_duplicates(subset='Fecha').sort_values('Fecha')
        df_completo = df_completo.reset_index(drop=True)  

    # Crear la figura
    fig = go.Figure()

    # Gráfica de datos reales
    fig.add_trace(go.Scatter(
        x=df_completo['Fecha'],
        y=df_completo['Actividad Real'],
        mode='lines',
        name='Actividad Real',
        line=dict(color='blue')
    ))
    
    # Gráfica de ciclo circadiano
    fig.add_trace(go.Scatter(
        x=df_completo['Fecha'],
        y=df_completo['Ciclo'],
        mode='lines',
        name='Ciclo',
        line=dict(color='green')
    ))

    # Gráfica de datos predichos
    fig.add_trace(go.Scatter(
        x=df_completo['Fecha'],
        y=df_completo['Prediccion'],
        mode='lines',
        name='Predicción',
        line=dict(color='red', dash='dot')
    ))
    # Gráfica ciclos predichos
    fig.add_trace(go.Scatter(
        x=df_completo['Fecha'],
        y=df_completo['Ciclo_p'],
        mode='lines',
        name='Ciclo_p',
        line=dict(color='grey', dash='dash')
    ))

    # Línea de separación
    fig.add_shape(
        dict(
            type="line",
            x0=fecha_separacion, y0=0, x1=fecha_separacion, y1=1,
            yref='paper', line=dict(color="black", dash="dash")
        )
    )
    
    agregar_anotaciones(fig, df_completo, horas_estres_f,'Prediccion','green')
    
    # Agregar anotaciones solo si el toggle está activado
    if activar_anotaciones:
        agregar_anotaciones(fig, df_completo, horas_estres_r, 'Actividad Real','blue')
    
    if activar_anotaciones2:
        agregar_anotaciones(fig, df_completo, horas_estres_pred,'Prediccion','red')
   
    
    
    # Configuración del layout
    fig.update_layout(
        # title="Nivel de Actividad y Predicción Continua",
        xaxis=dict(title="Línea de tiempo", rangeslider=dict(visible=True), type='date'),
        yaxis=dict(title="Actividad"),
        margin=dict(l=40, r=40, t=40, b=40)
    )
    return fig


# Ejecutar el servidor
if __name__ == '__main__':
    app.run_server(debug=True)


c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names



2024-12-30 00:00:00


C:\Users\PC\AppData\Local\Temp\ipykernel_20752\2617070546.py:69: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Horas con niveles de estrés leve y alto: [(Timestamp('2024-09-12 00:00:00'), 'Leve'), (Timestamp('2024-09-13 15:00:00'), 'Leve'), (Timestamp('2024-09-14 01:00:00'), 'Leve'), (Timestamp('2024-09-15 02:00:00'), 'Leve'), (Timestamp('2024-09-15 22:00:00'), 'Leve'), (Timestamp('2024-09-15 23:00:00'), 'Alto'), (Timestamp('2024-09-17 18:00:00'), 'Alto'), (Timestamp('2024-09-17 19:00:00'), 'Alto'), (Timestamp('2024-09-24 10:00:00'), 'Leve'), (Timestamp('2024-09-24 17:00:00'), 'Leve'), (Timestamp('2024-09-25 10:00:00'), 'Alto'), (Timestamp('2024-09-25 11:00:00'), 'Alto'), (Timestamp('2024-09-25 12:00:00'), 'Alto'), (Timestamp('2024-09-26 16:00:00'), 'Leve'), (Timestamp('2024-09-26 17:00:00'), 'Leve'), (Timestamp('2024-09-26 18:00:00'), 'Leve'), (Timestamp('2024-09-27 00:00:00'), 'Leve'), (Timestamp('2024-09-27 23:00:00'), 'Alto'), (Timestamp('2024-09-28 00:00:00'), 'Leve'), (Timestamp('2024-09-28 22:00:00'), 'Leve'), (Timestamp('2024-09-28 23:00:00'), 'Al

2024-12-23 00:00:00
2024-12-16 00:00:00
2024-12-09 00:00:00
2024-12-02 00:00:00
2024-11-25 00:00:00
2024-11-18 00:00:00
2024-11-10 00:00:00
2024-11-03 00:00:00
2024-10-23 00:00:00
2024-10-16 00:00:00
2024-10-09 00:00:00
2024-10-02 00:00:00
2024-09-25 00:00:00
2024-09-18 00:00:00
2024-09-11 00:00:00
2024-09-08 00:00:00
2024-09-08 00:00:00
2024-09-08 00:00:00
2024-09-08 00:00:00
2024-09-08 00:00:00
2024-09-08 00:00:00
2024-09-08 00:00:00
2024-09-08 00:00:00
2024-09-08 00:00:00


In [45]:
df_completo

,Fecha,Actividad Real,Ciclo,Prediccion,Ciclo_p
0,2024-12-16 00:00:00,-0.147244,-0.135078,0.001147,-0.089228
1,2024-12-16 01:00:00,-0.336617,-0.128951,-0.057128,-0.079561
2,2024-12-16 02:00:00,0.042130,-0.115247,0.142211,-0.065380
3,2024-12-16 03:00:00,-0.378878,-0.094901,-0.050708,-0.047652
4,2024-12-16 04:00:00,0.085171,-0.069299,-0.014028,-0.027584
...,...,...,...,...,...
427,2025-01-02 19:00:00,NaN,NaN,-0.115188,-0.003242
428,2025-01-02 20:00:00,NaN,NaN,-0.085513,-0.036048
429,2025-01-02 21:00:00,NaN,NaN,-0.076089,-0.069639
430,2025-01-02 22:00:00,NaN,NaN,-0.086313,-0.101724
